In [1]:
import pandas as pd
import folium
df = pd.read_csv('Dados_Bacia_8_Subestacao_86_Ativas.csv', sep=';')
df


,Altitude,Area_Drenagem,Bacia_Nome,Codigo_Adicional,Codigo_Operadora_Unidade_UF,Data_Periodo_Climatologica_Fim,Data_Periodo_Climatologica_Inicio,Data_Periodo_Desc_Liquida_Fim,Data_Periodo_Desc_liquida_Inicio,Data_Periodo_Escala_Fim,...,Tipo_Rede_Curso_Dagua,Tipo_Rede_Energetica,Tipo_Rede_Estrategica,Tipo_Rede_Navegacao,Tipo_Rede_Qual_Agua,Tipo_Rede_Sedimentos,UF_Estacao,UF_Nome_Estacao,codigobacia,codigoestacao
0,1048.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",83919,NaN,NaN,1948-01-01 00:00:00.0,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850001
1,1150.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,1.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850002
2,640.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850009
3,-1.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951147
4,-2.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,1.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,NaN,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850035
248,NaN,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850036
249,800.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,1952-04-01 00:00:00.0,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850013
250,960.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850016


In [2]:
# Converter a coluna 'Data_Periodo_Telemetrica_Inicio' para o formato datetime
df['Data_Periodo_Telemetrica_Inicio'] = pd.to_datetime(df['Data_Periodo_Telemetrica_Inicio'], errors='coerce')

# Filtrar as linhas onde a data é igual ou anterior a 01/01/2018
df = df[df['Data_Periodo_Telemetrica_Inicio'] <= '2018-01-01']
df = df[df['Tipo_Estacao_Telemetrica'] == 1]



In [4]:
import folium

# Supondo que o dataframe 'data' já está carregado no ambiente
data = df

# Filtrar as estações telemétricas
telemetricas = data[data['Tipo_Estacao_Telemetrica'] == 1]

# Classificar as estações telemétricas entre fluviométricas e pluviométricas usando a coluna 'Tipo_Estacao'
fluviometricas = telemetricas[telemetricas['Tipo_Estacao'] == 'Fluviometrica']
pluviometricas = telemetricas[telemetricas['Tipo_Estacao'] == 'Pluviometrica']

# Função para formatar o popup com informações adicionais
def format_popup(row):
    # Substituir NaN por 'Não disponível'
    rio_nome = row['Rio_Nome'] if pd.notna(row['Rio_Nome']) else 'Não disponível'
    area_drenagem = row['Area_Drenagem'] if pd.notna(row['Area_Drenagem']) else 'Não disponível'
    
    return (f"Estação: {row['Estacao_Nome']}<br>"
            f"Rio: {rio_nome}<br>"
            f"Código da Estação: {row['codigoestacao']}<br>"
            f"Área de Drenagem: {area_drenagem} km²")

# Criar o mapa base centrado em uma localização aproximada da bacia
mapa = folium.Map(location=[-29.5, -51.9], zoom_start=10)

# Criar grupos de camadas para seleção
pluviometricas_layer = folium.FeatureGroup(name="Pluviométricas", show=True)
fluviometricas_layer = folium.FeatureGroup(name="Fluviométricas", show=True)



# Adicionar estações pluviométricas ao grupo de camada
for _, row in pluviometricas.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=format_popup(row),
        icon=folium.Icon(color='green')
    ).add_to(pluviometricas_layer)
# Adicionar estações fluviométricas ao grupo de camada
for _, row in fluviometricas.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=format_popup(row),
        icon=folium.Icon(color='blue')
    ).add_to(fluviometricas_layer)

# Adicionar as camadas ao mapa
fluviometricas_layer.add_to(mapa)
pluviometricas_layer.add_to(mapa)

# Adicionar o controle de camadas (checkbox)
folium.LayerControl(collapsed=False).add_to(mapa)

# Adicionar uma legenda personalizada
legend_html = '''
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 250px; height: 100px;
    background-color: white; z-index:9999; font-size:14px;
    border:2px solid grey; padding: 10px;">
    <h4>Legenda</h4>
    <i class="fa fa-map-marker" style="color:green"></i> Pluviométricas<br>
    <i class="fa fa-map-marker" style="color:blue"></i> Fluviométricas<br>
    
</div>
'''
mapa.get_root().html.add_child(folium.Element(legend_html))

# Salvar o mapa em um arquivo HTML
mapa.save('mapa_estacoes_telemetricas.html')

print("mapa_estacoes_telemetricas.html'")


mapa_estacoes_telemetricas.html'


In [24]:
telemetricas

,Altitude,Area_Drenagem,Bacia_Nome,Codigo_Adicional,Codigo_Operadora_Unidade_UF,Data_Periodo_Climatologica_Fim,Data_Periodo_Climatologica_Inicio,Data_Periodo_Desc_Liquida_Fim,Data_Periodo_Desc_liquida_Inicio,Data_Periodo_Escala_Fim,...,Tipo_Rede_Curso_Dagua,Tipo_Rede_Energetica,Tipo_Rede_Estrategica,Tipo_Rede_Navegacao,Tipo_Rede_Qual_Agua,Tipo_Rede_Sedimentos,UF_Estacao,UF_Nome_Estacao,codigobacia,codigoestacao
2,640.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850009
3,-1.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951147
4,-2.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,1.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951148
5,NaN,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951150
6,1.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",DNIT,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,465.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850033
246,NaN,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850034
247,NaN,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850035
248,NaN,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850036


In [22]:
fluviometricas

,Altitude,Area_Drenagem,Bacia_Nome,Codigo_Adicional,Codigo_Operadora_Unidade_UF,Data_Periodo_Climatologica_Fim,Data_Periodo_Climatologica_Inicio,Data_Periodo_Desc_Liquida_Fim,Data_Periodo_Desc_liquida_Inicio,Data_Periodo_Escala_Fim,...,Tipo_Rede_Curso_Dagua,Tipo_Rede_Energetica,Tipo_Rede_Estrategica,Tipo_Rede_Navegacao,Tipo_Rede_Qual_Agua,Tipo_Rede_Sedimentos,UF_Estacao,UF_Nome_Estacao,codigobacia,codigoestacao
45,321.0,1721.0,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,2021-05-01 00:00:00.0,NaN,...,3.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,86497000
47,NaN,1785.0,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,2024-09-01 00:00:00.0,NaN,...,3.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,86497400
49,260.0,1820.0,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,1939-10-01 00:00:00.0,NaN,...,3.0,0,0.0,0.0,3.0,0.0,RS,RIO GRANDE DO SUL,8,86500000
50,NaN,1611.0,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,2024-07-01 00:00:00.0,NaN,...,3.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,86493000
52,NaN,1721.0,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,2024-06-01 00:00:00.0,NaN,...,3.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,86495500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,240.0,16000.0,"ATLÂNTICO, TRECHO SUDESTE",ANA,24.0,NaN,NaN,NaN,1940-01-01 00:00:00.0,NaN,...,2.0,0,0.0,1.0,3.0,1.0,RS,RIO GRANDE DO SUL,8,86510000
161,NaN,684.0,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,2013-05-01 00:00:00.0,NaN,...,3.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,86520100
163,NaN,2290.0,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,2009-02-01 00:00:00.0,NaN,...,3.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,86505500
165,NaN,2340.0,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,2009-02-01 00:00:00.0,NaN,...,3.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,86507000


In [23]:
pluviometricas

,Altitude,Area_Drenagem,Bacia_Nome,Codigo_Adicional,Codigo_Operadora_Unidade_UF,Data_Periodo_Climatologica_Fim,Data_Periodo_Climatologica_Inicio,Data_Periodo_Desc_Liquida_Fim,Data_Periodo_Desc_liquida_Inicio,Data_Periodo_Escala_Fim,...,Tipo_Rede_Curso_Dagua,Tipo_Rede_Energetica,Tipo_Rede_Estrategica,Tipo_Rede_Navegacao,Tipo_Rede_Qual_Agua,Tipo_Rede_Sedimentos,UF_Estacao,UF_Nome_Estacao,codigobacia,codigoestacao
2,640.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850009
7,772.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",431320101,21.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951120
8,52.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",432130301,21.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951135
9,78.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",432145101,21.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951136
10,16.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",432260804,21.0,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2951137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,465.0,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850033
246,NaN,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850034
247,NaN,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850035
248,NaN,NaN,"ATLÂNTICO, TRECHO SUDESTE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1,0.0,0.0,0.0,0.0,RS,RIO GRANDE DO SUL,8,2850036
